<a href="https://colab.research.google.com/github/bmill42/streaming-data/blob/main/Using_the_Spotify_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with the Spotify API

As we've learned, Spotify's technology and business model depend heavily on **metadata** about the music the company hosts and streams. While we've already seen how to download our own streaming data, Spotify also makes much of its other metadata available via an API, or Application Programming Interface.

An API is a system that connects multiple software programs, usually when one application needs to request data from another. Computers use many kinds of APIs, including ones that allow programs to communicate with the underlying operating system, but a common type used by many modern applications is a **Web API**, which lets an app request information from some external source on the internet.

APIs drive many of the web apps we use every day, and also allow programmers to interact with data sources in ways that can be more flexible than the standard app allows. For example, when you search a website like Travelocity or Kayak for flights or hotels, those sites are using APIs to request information from many different airlines or hotel chains in order to present you with all the search results in one place. The API provides some structure and limitations on what kinds of data can be requested: Kayak can't access Delta Airlines' data directly, but can make certain kinds of requests through the API and can expect certain kinds of data in response.

Similarly, until some recent changes, it was easy to use the Twitter API to create Twitter bots that could do things like post automatic weather updates, or respond to tweets containing particular words.

And from [Spotify's own documentation](https://developer.spotify.com/documentation/web-api), the "Spotify Web API enables the creation of applications that can interact with Spotify's streaming service, such as retrieving content metadata, getting recommendations, creating and managing playlists, or controlling playback." But the API also imposes limits on things like the rate at which an app can request data, which protects Spotify's servers from being overloaded (potentially by malicious attacks).


## Using the Spotify API in Python with spotipy

As with many modern web APIs, there is a convenient library that we can use to interact with Spotify metadata in Python. Behind the scenes, working with an API is similar to loading a webpage: your computer sends an HTTP request to `GET` some kind of information, and if the request has the correct format and credentials, Spotify sends back the information you asked for. The `spotipy` library handles all of this, so that we can just write Python code asking for things like lists of artists or albums and receive data in standard Python structures without worrying about how web communications work.

Since `spotipy` is not a package that comes with a typical Python installation but it's freely available online, so we just have to install it in our Colab session using `pip`, a standard Python package management tool:

In [ ]:
!pip install Spotipy

We'll go ahead and import pandas, spotipy, and one particular function required to authenticate our API requests.

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Spotify requires anyone interacting with the API to have a Spotify account, and to write code using `spotipy` we specifically need a Spotify Developer account. Spotify expects all API calls to be associated with a specific application, so we also have to use our Spotify Developer account to create an "app".

This is easier than it sounds and just requires clicking some button on the Spotify website. The instructions are listed under [**Step 1** of this `spotipy` tutorial](https://github.com/spotipy-dev/spotipy/blob/2.22.1/TUTORIAL.md#step-1-creating-a-spotify-account).

Once we've followed those steps, we should have two pieces of information, both available in the Settings page our "app" in the Spotify Developer portal:

- **Client ID**
- **Client Secret**

Each of these should be copied into the appropriate slot in the cell below. Running the cell sets up `spotipy` to make API calls that are authorized by your account and the "app" that you created.


In [ ]:
client_id = '391308ad998b4967ac275115233b66cf'
client_secret = 'd29ef3cd3d644037ae21b173708ca9d5'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Making API calls

Now we're ready to start talking to the Spotify API! We can think of the object `sp`, created in the previous cell, as the authorized user of our app - any code that asks for Spotify metadata will start with `sp.method_name()`.



### Taylor Swift's albums

For example, let's get a list of Taylor Swift's albums on Spotify.

First, we need to find Taylor Swift's **Spotify URI** (Uniform Resource Indicator). Every entity (artist, album, song, etc.) on Spotify has a URI that can identify it. The URIs look random, but they are static and permanent for any given entity.

To find Taylor's URI, navigate to [her page on Spotify](https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02) and look at the URL in your browser. Everything there that follows the last slash in the URL path is Taylor's URI, in this case: `06HL4z0CvFAxyc27GXpf02`

Run this code and look at the results, then we'll break down the steps:

In [ ]:
taylor_uri = '06HL4z0CvFAxyc27GXpf02'

results = sp.artist_albums(taylor_uri, album_type='album', limit=50)

albums = results['items']

for album in albums:
    print(album['name'])

The real action - the call to the API - happens in `sp.artist_albums()`.

We've given the method three arguments:
1. first is the URI for the artist
2. second is a specification of which type of album we want. The options, according to the `spotipy` [documentation](https://spotipy.readthedocs.io/en/2.24.0/#spotipy.client.Spotify.artist_albums) and the [Spotify API docs](https://developer.spotify.com/documentation/web-api/reference/get-an-album), are 'album', 'single', 'appears_on', and 'compilation'
3. Third is the `limit` for how many albums to return. If you leave this argument out, the default is 20, but I know Taylor Swift has more than that, so I asked for the max. [This example](https://github.com/spotipy-dev/spotipy/blob/2.22.1/TUTORIAL.md#step-3-start-using-spotipy) shows how to get the full list if an artist has more than 50 albums.

Now, let's zero in on the thing the API call returned, the `results`:

In [ ]:
results

Unfortunately, this printout is almost impossible to read because of those lengthy `available_markets` fields. Here's an excerpt with those removed:

```
{
    'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02/albums?offset=20&limit=20&include_groups=album',
    'limit': 50,
    'next': None,
    'offset': 0,
    'previous': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02/albums?offset=0&limit=20&include_groups=album',
    'total': 29,
    'items':
        [{
            'album_type': 'album',
            'total_tracks': 22,
            'available_markets': [],
            'external_urls': {'spotify': 'https://open.spotify.com/album/1KVKqWeRuXsJDLTW0VuD29'},
            'href': 'https://api.spotify.com/v1/albums/1KVKqWeRuXsJDLTW0VuD29',
            'id': '1KVKqWeRuXsJDLTW0VuD29',
            'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b273254c8a09649551438b20f4c0',
                    'height': 640,
                    'width': 640},
                {'url': 'https://i.scdn.co/image/ab67616d00001e02254c8a09649551438b20f4c0',
                    'height': 300,
                    'width': 300},
                {'url': 'https://i.scdn.co/image/ab67616d00004851254c8a09649551438b20f4c0',
                    'height': 64,
                    'width': 64}],
            'name': 'Red (Deluxe Edition)',
            'release_date': '2012-10-22',
            'release_date_precision': 'day',
            'type': 'album',
            'uri': 'spotify:album:1KVKqWeRuXsJDLTW0VuD29',
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
                'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
                'id': '06HL4z0CvFAxyc27GXpf02',
                'name': 'Taylor Swift',
                'type': 'artist',
                'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
            'album_group': 'album'
        },
        {
            'album_type': 'album',
            'total_tracks': 16,
   ...
  }]
}
   ```

This data is in the form of a Python **dictionary**, which looks just like the JSON we saw previously, including the structure of `key:value` pairs.

The first few rows are metadata about *all* of Taylor Swift's albums:
- the link to her albums page on Spotify (`href`)
- the `limit` for how many albums to include, which I set to the max of 50
- some information that can be used to get more albums than the limit allows (`next, offset`, etc., irrelevant for now)

The important thing here is the next key: `items`. The *value* for `items` is *itself a list* containing information for each individual album.

### Navigating Python dictionaries

Let's step back and look at a simple dictionary:

In [ ]:
test_dict = {
    'artist': 'Taylor Swift',
    'total_albums': 29,
    'album_names': [
        '1989',
        'Red',
        'Midnights']
}

We can access the values stored in dictionaries by giving the dictionary name with square brackets and the name of a key, as in `dict_name['key']`. Note that keys are often strings, which means they need quotes around them.

In [ ]:
test_dict['artist']

A key can take a simple value, like a number or a string, but it can also take a larger data structure, like a list. That's the case here for the `album_names` key:

In [ ]:
test_dict['album_names']

Since the thing returned by that last line of code is itself a list, we can index into it to get its contents, even though the resulting code looks a little odd. Let's get the first item (number `0` in Python terms) from the list of album names:

In [ ]:
test_dict['album_names'][0]

The `results` dictionary from our Taylor Swift query goes one step further: the value associated with the key `items` is *another dictionary*. This explains why one line of code in the API call example is

```
albums = results['items']
```

We're looking inside the `results` dictionary and grabbing the value associated with `items`, which is itself a list of dictionaries.

Finally, each individual album dictionary has keys for things like `name`, `release_date`, etc. Our code just loops through the items in the list of `albums` and prints out the name, but we can easily modify this to include additional information, like the release date:

In [ ]:
for album in albums:
    print(album['release_date'] + " : " + album['name'])

## Tracks and audio features

In addition to the typical metadata we've seen so far - album names and release dates, track names and durations, etc. - we can also access a more proprietary set of metadata: Spotify's **audio features**, which consist of rating for each track that attempt to capture things like "danceability", "loudness", "acousticness", and more. The full list of features can be found in the [API documentation](https://developer.spotify.com/documentation/web-api/reference/get-audio-features).

Let's look at the features for Radiohead's 1995 track, "High and Dry". The variable `tid` stands for "track ID":

In [ ]:
tid = '2a1iMaoWQ5MnvLFBDv4qkf' # Track ID from the URL for 'High and Dry'

high_dry_features = sp.audio_features(tid)
high_dry_features

Each of these features is calculated using some kind of audio analysis algorithm, which we don't have access to, and we should take all of them with a grain of salt - Spotify doesn't even really specify what something like "acousticness" means in the documentation!

That said, these features are part of what Spotify uses to make recommendations, so they play an important role in today's musical ecosystem even if they are rather abstract and hidden from users. A sign of how valuable Spotify finds these features is that their documentation specifies very clearly that users are NOT allowed to use this information to train their own machine learning models.

For now, let's just manually compare "High and Dry" with another Radiohead track, 2000's "Idioteque":

In [ ]:
tid = '1W5XugQJGhnSATMI5n002M' # Track ID from the URL for 'Idioteque'

idioteque_features = sp.audio_features(tid)
idioteque_features

The danceability is higher, which seems to make sense given the track's driving beat, and the acousticness is a bit lower, which seems to fit the track's electronic sounds (though "High and Dry" also has fairly low acousticness, which is perhaps surprising).

### Features for multiple tracks

Reading these JSON outputs is not really the best way to compare tracks. Let's get all the tracks from a single album and work with the results as a dataframe instead.

We'll collect audio features for all the tracks from T-Pain's album *Epiphany* (2007), then walk through the details:

In [ ]:
album_id = '6CrSEKCF4TYrbSIitegb3h' # Album URI for Epiphany

results = sp.album_tracks(album_id)

tracks = results['items']
track_ids = [track['id'] for track in tracks]
track_names = [track['name'] for track in tracks]

audio_features = sp.audio_features(track_ids)
audio_features

As always, we create a `results` variable that stores the result of an API call, in this case `album_tracks()`, with the URI for the album we're interested in.

And as in other examples, the information we care about is nested inside the `results`, attached to a key called `items`, so we can get a list of the tracks' metadata with `tracks = results['items']`.

Each item in the list of `tracks` is another dictionary with information on a single track. The key `id` contains the URI for the track, and the key `name` contains the track name.

Lines 6 and 7 use a coding trick called a *list comprehension* to create lists containing only the IDs or names of each track. The syntax: `[X for Y in Z]` is just shorthand for a `for` loop - try running the following two cells and notice how they produce the same output:

In [ ]:
track_names = []

for track in tracks:
    track_names.append(track['name'])

track_names

In [ ]:
track_names = [track['name'] for track in tracks]
track_names

### From a list of features to a dataframe

Now let's take that awkward JSON format and turn it into a dataframe. Just like we did with the Spotify listening data, we can just call `pd.DataFrame()` with the dictionary as an argument:

In [ ]:
df = pd.DataFrame(audio_features)
df.head()

But this mostly consists of numbers and, importantly, doesn't include track names! Luckily, we already have a list of the track names, created using the loop shortcut above.

Let's create a new column and set it to contain the list of track names. And to simplify the dataframe and ignore those unhelpful URLs, let's reduce it down to only the columns that we care about:

In [ ]:
df['track_name'] = track_names
df = df[['track_name', 'danceability', 'energy', 'speechiness', 'acousticness',
         'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
df

### The complete example

Here is all the code from that example in one place:

In [ ]:
album_id = '6CrSEKCF4TYrbSIitegb3h' # Album URI for Epiphany

results = sp.album_tracks(album_id)

tracks = results['items']
track_ids = [track['id'] for track in tracks]
track_names = [track['name'] for track in tracks]

audio_features = sp.audio_features(track_ids)

df['track_name'] = track_names
df = df[['track_name', 'danceability', 'energy', 'speechiness', 'acousticness',
         'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]

# Key terms

For this lab, the key concepts are:

- **Spotify URIs**: Every artist, album, and track on Spotify has a uniform record identifier (URI) that can be found by searching for the appropriate page on the Spotify website and copying the random-looking letters that appear in the URL
- **APIs and API calls**: once our credentials are set up, all we have to do is use a `spotipy` API method like `sp.album_tracks()` or `sp.audio_features()` to get data from Spotify
- **Dictionaries**: Data from the API comes in the form of a Python dictionary, so we need to understand how to index dictionaries using `dict[key]` to get the *value* for that key
 - What makes this tricky is that Spotify's data dictionaries are often large and deep---sometimes their values are themselves more dictionaries, not just strings or numbers

FYI, the developers of `spotipy` maintain a [list of examples of API use cases, with Python code](https://github.com/spotipy-dev/spotipy/tree/master/examples), from getting lists of tracks and albums to manipulating a user's playlists and asking Spotify for algorithmic recommendations.

# Exercise

1. Choose an album that you like, find its Spotify URI, and use the code from the last example to make an API call requesting its audio feature data. (Optionally, you can try something more complex, like getting the feature data for a set of tracks by different artists)

2. As in the example, turn the data dictionary into a dataframe with track names.

3. Then, export the data as a CSV or Excel file using `pd.to_csv` or `pd.to_excel`, load that file in Google Sheets, and create one or more visualizations of the data.

4. Write a short description of something interesting you find in the feature data. Is there a particular trajectory across the course of the album for one or more of the features? Compared to your knowledge of how the actual music sounds, are there any audio features that don't make sense or seem to miss something about the music? (say, a track with a high danceability score that isn't actually danceable)

Share your Google Sheet with me, with your name in the title, and submit the link to the Canvas assignment. Your short writeup can be included in the spreadsheet or submitted separately. You do NOT need to share the notebook with the code unless you want to.

In [ ]:
# Your code here